In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import wandb


In [ ]:
BATCH_SIZE = 64
NUM_CLASSES = 7
LEARNING_RATE = 0.001
EPOCHS_NUM = 15

wandb.init(
    project="efficientnet-skin-disease-model",
    config={
        "batch_size": BATCH_SIZE,
        "num_classes": NUM_CLASSES,
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS_NUM,
    }
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



wandb: Currently logged in as: tetiana-trachuk-kn-2021 (tetiana-trachuk-kn-2021-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



In [ ]:
dataset = ImageFolder(root='C:/Diploma/classification_dataset', transform=augmentation_transforms)
train_size = int(0.85 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
test_dataset.dataset.transform = test_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Load Pretrained EfficientNet Model
model = models.efficientnet_b0(pretrained=True)
in_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_features, NUM_CLASSES)
)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)


c:\Diploma\Diploma\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Diploma\Diploma\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
for epoch in range(EPOCHS_NUM):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)
    
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    
    print(f"Epoch [{epoch+1}/{EPOCHS_NUM}], Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.4f}, Prec: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    wandb.log({
        "epoch": epoch+1, 
        "loss": running_loss/len(train_loader), 
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

Epoch [1/15], Loss: 0.9050, Acc: 0.6444, Prec: 0.6082, Recall: 0.6037, F1: 0.6055
Epoch [2/15], Loss: 0.6502, Acc: 0.7506, Prec: 0.7232, Recall: 0.7217, F1: 0.7221
Epoch [3/15], Loss: 0.5083, Acc: 0.8066, Prec: 0.7821, Recall: 0.7814, F1: 0.7816
Epoch [4/15], Loss: 0.4097, Acc: 0.8503, Prec: 0.8304, Recall: 0.8306, F1: 0.8303
Epoch [5/15], Loss: 0.3292, Acc: 0.8804, Prec: 0.8642, Recall: 0.8643, F1: 0.8642
Epoch [6/15], Loss: 0.2755, Acc: 0.8994, Prec: 0.8858, Recall: 0.8857, F1: 0.8857
Epoch [7/15], Loss: 0.2268, Acc: 0.9193, Prec: 0.9069, Recall: 0.9073, F1: 0.9070
Epoch [8/15], Loss: 0.1756, Acc: 0.9385, Prec: 0.9293, Recall: 0.9306, F1: 0.9299
Epoch [9/15], Loss: 0.1796, Acc: 0.9355, Prec: 0.9258, Recall: 0.9261, F1: 0.9260
Epoch [10/15], Loss: 0.1363, Acc: 0.9501, Prec: 0.9418, Recall: 0.9429, F1: 0.9423
Epoch [11/15], Loss: 0.1474, Acc: 0.9464, Prec: 0.9398, Recall: 0.9392, F1: 0.9395
Epoch [12/15], Loss: 0.1192, Acc: 0.9574, Prec: 0.9515, Recall: 0.9522, F1: 0.9518
Epoch [13/15]

In [7]:
torch.save(model.state_dict(), "efficientnet_skin_disease_model.pth")
print("Model training complete and saved.")

Model training complete and saved.
